In [ ]:
#initial commit

In [ ]:
#Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Understanding

Data Exploration

In [ ]:
#load the dataset
dataset=pd.read_csv('/content/restaurant_customer_satisfaction.csv')

In [ ]:
#View Dataset
dataset

In [ ]:
#viewing first five rows of the dataset
dataset.head()

In [ ]:
#Viewing last three raws of the dataset
dataset.tail(3)

In [ ]:
dataset.loc[0:5]

In [ ]:
#viewing data raws upto zero to twenty 3 by 3
dataset.loc[0:20:3]

In [ ]:
#viewing the dimention of the dataset
dataset.shape

In [ ]:
#Viewing the coloumns names of the dataset
dataset.columns

In [ ]:
#viewing dataset as a dataframe
df=pd.DataFrame(dataset)
df

In [ ]:
#get the detailed informations of the features
df.info()

In [ ]:
#viewing specific colomn only
df['Income']

In [ ]:
#viewing specific rows of a specific colomn
df.loc[0:1,'Income']

In [ ]:
df[["Age", "Gender", "MealType"]]

In [ ]:
#get the mean of FoodRating colomn
meann = df['FoodRating'].mean()
meann

In [ ]:
#Round the mean in to two points
round(meann,2)

In [ ]:
#Viewing the data types of the colomns
dataset.dtypes

In [ ]:
#Viewing the total number of null values in the data set according to the features
dataset.isnull().sum()

In [ ]:
# drop unwanted columns
dataset = dataset.drop(['CustomerID','Gender', 'GroupSize', 'MealType', 'OnlineReservation', 'DeliveryOrder', 'WaitTime', 'AverageSpend'],axis=1)

In [ ]:
#Viewing the dataset after dropping the unwanted colomns
dataset.columns

In [ ]:
#Get the statistical description of the data(only Numarical data will describe)
dataset.describe()

## Data Preprocessing

In [ ]:
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#get a copy of dataset
original_dataset = dataset.copy()

In [ ]:
#differntiate the data(numarical/catergorical/bianary/non-binary)
non_binary_categorical_columns =['VisitFrequency', 'PreferredCuisine', 'TimeOfVisit', 'DiningOccasion']
numarical_columns = ['Age', 'Income',  'LoyaltyProgramMember', 'ServiceRating', 'FoodRating', 'AmbianceRating','HighSatisfaction']

In [ ]:
#Apply one-hot encoding
VisitFrequency_dummie = pd.get_dummies(original_dataset['VisitFrequency'], prefix = 'VisitFrequency')
PreferredCuisine_dummie = pd.get_dummies(original_dataset['PreferredCuisine'], prefix = 'PreferredCuisine')
TimeOfVisit_dummie = pd.get_dummies(original_dataset['TimeOfVisit'], prefix = 'TimeOfVisit')
DiningOccasion_dummie = pd.get_dummies(original_dataset['DiningOccasion'], prefix = 'DiningOccasion')

In [ ]:
#drop the non-binary categorical colomns from the copy of dataset and stote in another variable
new_data = original_dataset.drop(non_binary_categorical_columns, axis=1)

In [ ]:
#concatenate the dummies
OH_data = pd.concat([new_data,VisitFrequency_dummie,PreferredCuisine_dummie,TimeOfVisit_dummie,DiningOccasion_dummie], axis=1)
print (OH_data)


OH_data.columns = OH_data.columns.astype(str)

OH_data.head()

In [ ]:
#get a copy of encoded data and check the null values
encoded_data = OH_data.copy()
encoded_data.isnull().sum()

## Data Visualisation

In [ ]:
# Plot distributions for numerical features
numeric_columns = ['Income', 'HighSatisfaction',  'LoyaltyProgramMember', 'ServiceRating', 'FoodRating', 'AmbianceRating']
for column in numeric_columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[column].dropna(), kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

In [ ]:

sns.displot(encoded_data["Income"])


In [ ]:
sns.displot(encoded_data["HighSatisfaction"])

In [ ]:
sns.displot(encoded_data["LoyaltyProgramMember"])

In [ ]:
#plotting the boxplots of some features
plt.boxplot(encoded_data['ServiceRating'])
plt.title('Service Rating Explot')
plt.ylabel('Service Rating')
plt.show()

In [ ]:
plt.boxplot(encoded_data['FoodRating'])
plt.title('Food Rating Explot')
plt.ylabel('Food Rating')
plt.show()

In [ ]:
plt.boxplot(encoded_data['AmbianceRating'])
plt.title('Ambiance Rating Explot')
plt.ylabel('AmbianceRating')
plt.show()